###### resource: https://github.com/Applied-Language-Technology/notebooks/blob/main/part_iii/05_embeddings_continued.ipynb

#### **E. Contextual Embeddings**

So far, the embeddings that we have learned are described as `static`. This means that the vector representation for a given token **regardless** of the `context` in which the token occurs.

Words can have more than one meaning depending on the context they appear:

1. Manila is the `capital` of the Philippines.
2. The most important advice anyone can receive in trading the market is risk management - protect your `capital`, profits just follow.

Static word embeddings cannot capture this distinction because words will always have the same vector representation. The values of the `n-dimensional` vector remains the same.

We would like to encode the context by which a word occurs to draw out these differences in meaning, using an architecture called **`Transformers`**.

A **`Transformer`** is a neural network architecture that is capable of encoding information about the context which a token occurs. It improved performance for traditional natural language processing tasks such as part-of-speech tagging and syntactic parsing, but it is `computationally heavy`.

In [1]:
import spacy

In [2]:
# Load a Transformer-based language model
nlp_trf = spacy.load('en_core_web_trf')

On the surface, a *Language* object that contains a Transformer-based model looks and works just like any other language model in spaCy.

However, if we look under the hood of the *Language* object under `nlp_trf` using `pipeline`, we will see that the first component in the processing pipeline is a Transformer.

In [3]:
nlp_trf.pipeline

[('transformer',
  <spacy_transformers.pipeline_component.Transformer at 0x1fe0b8c4b20>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x1fe0b8c4f40>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x1fe0b71bf20>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x1fe0b90fa80>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x1fe0b911840>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x1fe0b8ff120>)]

In [8]:
# Feed an example sentence to the model
example_doc = nlp_trf('The most important advice anyone can receive in trading the market is risk management - protect your capital, profits just follow ~ Paul Aldrin Enriquez')

# Check the length of the document
# It counts the punctuation marks
len(example_doc)

26

The output of the transformer is stored in an object in spaCy called `trf_data`.

The `tensors` attribute of the *TransformerData* object contains a list with vector representations generated by the Transformer for individual *Tokens* and the entire *Doc* object.

The first item in the `tensors` list under index 0 contains the output for the individual tokens.

In [9]:
example_doc._.trf_data.tensors[0].shape

(1, 31, 768)

This means there is one batch of `31` vectors, each with `768` dimensions.

In [13]:
# Check the first ten dimensions of the tensor
example_doc._.trf_data.tensors[0][0][:10]

array([[ 0.3221337 ,  0.03971476, -1.6743078 , ...,  0.31791827,
         0.4955263 , -0.5075195 ],
       [-0.6849789 , -0.2843948 ,  0.740689  , ..., -1.0440093 ,
         0.07250463, -0.9174782 ],
       [ 0.9529375 ,  0.42180288, -0.4646021 , ..., -0.01987644,
        -0.69030166,  1.3408207 ],
       ...,
       [ 0.59506214, -1.7604082 ,  0.11381152, ..., -2.634387  ,
         0.59920263,  0.68672407],
       [-0.6051442 , -0.01173162,  0.52655953, ...,  0.20555082,
         0.08978168, -1.4721365 ],
       [ 0.39855224, -1.3170516 , -0.554474  , ..., -1.1178719 ,
        -0.03808063,  0.48295164]], dtype=float32)

The second item under index 1 holds the output for the entire *Doc* object.

In [10]:
example_doc._.trf_data.tensors[1].shape
# A single vector with 768 dimensions

(1, 768)

The Transformers object output 24 vectors for a spaCy Doc object of 22 tokens. They don't match.

Vocabulary size of all words just in the English language alone is massive, and to learn every unique word would blow up the size of the model.

Because Transformers are trained on massive volumes of text, the model's vocabulary must be kept limited.

To address the issue, Transformers use more complex tokenizers that identify frequently occuring character sequences in the data and learn embeddings for these sequences instead. These sequences, which are often referred as `subwords`, make up the vocabulary of the Transformer.

This information is stored in the attribute `tokens`, which contains a dictionary of subwords under the key `input_texts`.

In [11]:
example_doc._.trf_data.tokens['input_texts']

[['<s>',
  'The',
  'Ġmost',
  'Ġimportant',
  'Ġadvice',
  'Ġanyone',
  'Ġcan',
  'Ġreceive',
  'Ġin',
  'Ġtrading',
  'Ġthe',
  'Ġmarket',
  'Ġis',
  'Ġrisk',
  'Ġmanagement',
  'Ġ-',
  'Ġprotect',
  'Ġyour',
  'Ġcapital',
  ',',
  'Ġprofits',
  'Ġjust',
  'Ġfollow',
  'Ġ~',
  'ĠPaul',
  'ĠAld',
  'rin',
  'ĠEn',
  'ri',
  'quez',
  '</s>']]

Here we can see how the document has been tokenized. They begin and terminate with the tag `<s>`, and the `Ġ` indicates space. The more interesting part is how my name has been tokenized. 
`'ĠPaul',`
  `'ĠAld',`
  `'rin',`
  `'ĠEn',`
  `'ri',`
  `'quez'`

This is so because my name is not in the Transformer vocabulary, but the subwords of my name are. Their individual vector representations will be used to construct the vector for each word of my name.

This is done for optimization. The subwords that make up the vocabulary are those that occur frequently in the training data the Transformer is trained with.

To map these vectors to `Tokens` in the spaCy Doc object, we must retrieve alignment information from the `align` attribute of the TransformerData object.

The `align` attribute can be indexed using the indices of Token objects in the Doc object.

To exemplify, we can retrieve the last Token "Enriquez" in the Doc object doc using the expression `example_doc[0]`.

We then use the index of this Token in the Doc object to retrieve alignment data, which is stored under the `align` attribute.

More specifically, we need the information stored under the attribute `data`.

In [14]:
#Get the last spaCy Token, "Enriquez", and its alignment data
example_doc[-1], example_doc._.trf_data.align[-1].data

(Enriquez,
 array([[27],
        [28],
        [29]]))

In this case, vectors at indices 27, 28 and 29 in the batch of 31 vectors contain the representation for "Enriquez".

To use the contextual embeddings from the Transformer efficiently, we can define a component that retrieves contextual word embeddings for Docs, Spans and Tokens and add this component to the spaCy pipeline.

This can be achieved by creating a new Python Class. Because the new Class will become a component of the spaCy pipeline, we must first import the Language object and let spaCy know that we are now defining a new pipeline component.

In [15]:
# Import the Language object under the 'language' module in spaCy,
# and NumPy for calculating cosine similarity.
from spacy.language import Language
import numpy as np

# We use the @ character to register the following Class definition
# with spaCy under the name 'tensor2attr'.
@Language.factory('tensor2attr')

# We begin by declaring the class name: Tensor2Attr. The name is 
# declared using 'class', followed by the name and a colon.
class Tensor2Attr:
    
    # We continue by defining the first method of the class, 
    # __init__(), which is called when this class is used for 
    # creating a Python object. Custom components in spaCy 
    # require passing two variables to the __init__() method:
    # 'name' and 'nlp'. The variable 'self' refers to any
    # object created using this class!
    def __init__(self, name, nlp):
        
        # We do not really do anything with this class, so we
        # simply move on using 'pass' when the object is created.
        pass

    # The __call__() method is called whenever some other object
    # is passed to an object representing this class. Since we know
    # that the class is a part of the spaCy pipeline, we already know
    # that it will receive Doc objects from the preceding layers.
    # We use the variable 'doc' to refer to any object received.
    def __call__(self, doc):
        
        # When an object is received, the class will instantly pass
        # the object forward to the 'add_attributes' method. The
        # reference to self informs Python that the method belongs
        # to this class.
        self.add_attributes(doc)
        
        # After the 'add_attributes' method finishes, the __call__
        # method returns the object.
        return doc
    
    # Next, we define the 'add_attributes' method that will modify
    # the incoming Doc object by calling a series of methods.
    def add_attributes(self, doc):
        
        # spaCy Doc objects have an attribute named 'user_hooks',
        # which allows customising the default attributes of a 
        # Doc object, such as 'vector'. We use the 'user_hooks'
        # attribute to replace the attribute 'vector' with the 
        # Transformer output, which is retrieved using the 
        # 'doc_tensor' method defined below.
        doc.user_hooks['vector'] = self.doc_tensor
        
        # We then perform the same for both Spans and Tokens that
        # are contained within the Doc object.
        doc.user_span_hooks['vector'] = self.span_tensor
        doc.user_token_hooks['vector'] = self.token_tensor
        
        # We also replace the 'similarity' method, because the 
        # default 'similarity' method looks at the default 'vector'
        # attribute, which is empty! We must first replace the
        # vectors using the 'user_hooks' attribute.
        doc.user_hooks['similarity'] = self.get_similarity
        doc.user_span_hooks['similarity'] = self.get_similarity
        doc.user_token_hooks['similarity'] = self.get_similarity
    
    # Define a method that takes a Doc object as input and returns 
    # Transformer output for the entire Doc.
    def doc_tensor(self, doc):
        
        # Return Transformer output for the entire Doc. As noted
        # above, this is the last item under the attribute 'tensor'.
        # Average the output along axis 0 to handle batched outputs.
        return doc._.trf_data.tensors[-1].mean(axis=0)
    
    # Define a method that takes a Span as input and returns the Transformer 
    # output.
    def span_tensor(self, span):
        
        # Get alignment information for Span. This is achieved by using
        # the 'doc' attribute of Span that refers to the Doc that contains
        # this Span. We then use the 'start' and 'end' attributes of a Span
        # to retrieve the alignment information. Finally, we flatten the
        # resulting array to use it for indexing.
        tensor_ix = span.doc._.trf_data.align[span.start: span.end].data.flatten()
        
        # Fetch Transformer output shape from the final dimension of the output.
        # We do this here to maintain compatibility with different Transformers,
        # which may output tensors of different shape.
        out_dim = span.doc._.trf_data.tensors[0].shape[-1]
        
        # Get Token tensors under tensors[0]. Reshape batched outputs so that
        # each "row" in the matrix corresponds to a single token. This is needed
        # for matching alignment information under 'tensor_ix' to the Transformer
        # output.
        tensor = span.doc._.trf_data.tensors[0].reshape(-1, out_dim)[tensor_ix]
        
        # Average vectors along axis 0 ("columns"). This yields a 768-dimensional
        # vector for each spaCy Span.
        return tensor.mean(axis=0)
    
    # Define a function that takes a Token as input and returns the Transformer
    # output.
    def token_tensor(self, token):
        
        # Get alignment information for Token; flatten array for indexing.
        # Again, we use the 'doc' attribute of a Token to get the parent Doc,
        # which contains the Transformer output.
        tensor_ix = token.doc._.trf_data.align[token.i].data.flatten()
        
        # Fetch Transformer output shape from the final dimension of the output.
        # We do this here to maintain compatibility with different Transformers,
        # which may output tensors of different shape.
        out_dim = token.doc._.trf_data.tensors[0].shape[-1]
        
        # Get Token tensors under tensors[0]. Reshape batched outputs so that
        # each "row" in the matrix corresponds to a single token. This is needed
        # for matching alignment information under 'tensor_ix' to the Transformer
        # output.
        tensor = token.doc._.trf_data.tensors[0].reshape(-1, out_dim)[tensor_ix]

        # Average vectors along axis 0 (columns). This yields a 768-dimensional
        # vector for each spaCy Token.
        return tensor.mean(axis=0)
    
    # Define a function for calculating cosine similarity between vectors
    def get_similarity(self, doc1, doc2):
        
        # Calculate and return cosine similarity
        return np.dot(doc1.vector, doc2.vector) / (doc1.vector_norm * doc2.vector_norm)

With the Class Tensor2Attr defined, we can now add it to the pipeline by referring to the name we registered with spaCy using `@Language.factory()`, that is, `tensor2attr`.

In [16]:
# Add the component named 'tensor2attr', which we registered using the
# @Language decorator and its 'factory' method to the pipeline.
nlp_trf.add_pipe('tensor2attr')

# Call the 'pipeline' attribute to examine the pipeline
nlp_trf.pipeline

[('transformer',
  <spacy_transformers.pipeline_component.Transformer at 0x1fe0b8c4b20>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x1fe0b8c4f40>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x1fe0b71bf20>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x1fe0b90fa80>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x1fe0b911840>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x1fe0b8ff120>),
 ('tensor2attr', <__main__.Tensor2Attr at 0x1fe0d8d0dc0>)]

This component stores the Transformer-based contextual embeddings for Docs, Spans and Tokens under the vector attribute.

Let's explore contextual embeddings by defining two Doc objects and feeding them to the Transformer-based language model under `nlp_trf`.

In [17]:
# Define two example sentences and process them using the Transformer-based
# language model under 'nlp_trf'.
doc_city_trf = nlp_trf("Manila is the capital of the Philippines.")
doc_money_trf = nlp_trf("The most important advice anyone can receive in trading the market is risk management - protect your capital, profits just follow.")

The noun `"capital"` has two different meanings in these sentences: in doc_city_trf, `"capital"` refers to a `city`, whereas in doc_money_trf the word refers to `money`.

The Transformer should encode this difference into the resulting vector based on the context in which the word occurs.

Let's fetch the Token corresponding to `"capital"` in each example and retrieve their vector representations under the `vector` attribute.

In [20]:
# Retrieve vectors for the two Tokens corresponding to "capital";
# assign to variables 'city_trf' and 'money_trf'.
city_trf = doc_city_trf[3]
money_trf = doc_money_trf[17]

# Compare the similarity of the two meanings of 'capital'
city_trf.similarity(money_trf)

0.64822173

Cosine similarity is `less than 1`, which means that their vector representations even they are the same word `capital` are different. This is because the Transformer also encodes information abouth their context of occurence into the vectors, which has allowed it to learn that the same linguistic form may have different meanings in different contexts.

This stands in stark contrast to `static` word embeddings, as shown below using a large language model for English in spaCy `en_core_web_lg`.

In [22]:
# Define two example sentences and process them using the large language model
# en_core_web_lg is a large language model for English, which contains word vectors for 685 000 Token objects.

# Load a large language model and assign it to the variable 'nlp_lg'
nlp_lg = spacy.load('en_core_web_lg')
doc_city_lg = nlp_lg("Manila is the capital of the Philippines.")
doc_money_lg = nlp_lg("The most important advice anyone can receive in trading the market is risk management - protect your capital, profits just follow.")

# Retrieve vectors for the two Tokens corresponding to "capital";
# assign to variables 'city_lg' and 'money_lg'.
city_lg = doc_city_lg[3]
money_lg = doc_money_lg[17]

# Compare the similarity of the two meanings of 'capital'
city_lg.similarity(money_lg)

1.0

As you can see, the vectors for the word `"capital"` are identical `(cosine similarity of 1)`, because the word embeddings do not encode information about the context in which the word occurs.

#### **End. Thank you!**